# Interactive Notebook for Calculating the return period from ensemble forecasts

In [90]:
from ipywidgets import interact
from netCDF4 import Dataset
import os
import numpy as np
import iris
import iris.quickplot as qplt
import matplotlib.pyplot as plt

## Defining various functions

In [32]:
def read_ensemble_member(file, varname):
    '''
    Reads in the forecast ensemble netcdf files
    file: path to the ensemble forecast file 
    varname: variable name to read in (amount_of_precipitation)
    '''
    data = Dataset(file)[varname]
    return data

In [58]:
def sliding_time(data):
    '''
    Calculates the maximum rainfal over sliding 24 hour windows of the
    54 hour lead time forecast
    
    data: precipitation array at all the time steps
    '''
    max_value = data[0,:,:]
    summed_value = data[0,:,:]
    for t in range((len(data[:,0,0])-24)):
        summed_value = np.sum(data[t:t+24,:,:], axis=0)
        max_value = np.maximum(summed_value[:,:], max_value[:,:])

    return max_value

In [64]:
def return_period(period, max_value, times):
    '''
    Compares the maximum rainfall over the sliding 24 hour window against the rainfall thresholds
    for different return periods at each coordinate position
    
    period: zero array which will store the integer value of the return period threshold which has been met
    max_value: The calculated maximum rainfall over the sliding 24 hour windows in the forecast
    times: This can be the 5,10,25,50,100 Return period
    '''
    for year in times:
        ERA = os.path.join("../Return_Periods_ERA5/regridded",str("ERA_RP_"+str(year)+".nc"))
        var = "r"+str(year)+"yrrp"
        return_estimate = read_ERA(ERA, var)
        max_value = np.float32(max_value)
        for x in range(len(max_value[:,0])):
            for y in range(len(max_value[0,:])):
                if return_estimate[x,y] < 10000.0:
                    if max_value[x,y] > return_estimate[x,y]:
                        period[x,y] = year
    return period

In [65]:
def read_ERA(file, varname):
    '''
    Read in the ERA return netCDF file
    file: path to the file
    varname: variable to be read in ("r_%year_yrrp")
    '''
    v = Dataset(file)[varname]
    data = v[:,:]
    return data

In [66]:
def read_coords(file, lonname, latname):
    '''
    Read in the projected x and y coordinates
    file: path to the file
    lonname: variable name
    latname: variable name
    '''
    v = Dataset(file)
    x = v[lonname]
    y = v[latname]
    return x, y

In [101]:
def write2nc(return_period_arr, x, y, outputfile):
    '''
    Write calculated return period to a netCDF file
    return_period_arr: return period array
    x: X coordinate
    y: Y coordinate
    outputfile: File name for output
    '''
    xname = "projection_x_coordinate"
    yname = "projection_y_coordinate"
    
    dataset_new = Dataset(outputfile, 'w', format='NETCDF4_CLASSIC')

    proj_y = dataset_new.createDimension(yname, len(y))
    proj_x = dataset_new.createDimension(xname, len(x))
    
    y_corr = dataset_new.createVariable(yname, np.float64,yname)
    x_corr = dataset_new.createVariable(xname, np.float64,xname)

    y_corr.units = 'm'
    x_corr.units = 'm'
    y_corr[:] = y
    x_corr[:] = x
    
    return_period = dataset_new.createVariable("return_period", np.float64, (yname,xname))
    return_period.units = "years"
    return_period.grid_mapping = "transverse_mercator"
    return_period[:] = return_period_arr
    
    # Close the netcdf file
    dataset_new.close()

    return

In [ ]:
def calculating_return_periods(Scenario):
    '''
    Calculate the spatially return priod for the ensemble forecasts
    Scenario: Ensemble member number
    
    '''
    # Return periods
    times = [5,10,25,50,100]
    # Set up some local paths and fixed variables
    base = "../ForecastData/ensemble_files"
    varname = "amount_of_precipitation"
    # Load in the ensemble member
    filename = str("202002080300_u1096_ng_ek"+ str(Scenario).zfill(2)+"_precipaccum_2km.nc")
    file = os.path.join(base,filename)
    data = read_ensemble_member(file, varname)
    # # Calculating the accumlated rain fall over 24 hour windows
    # # Each forecast has a 54-hour lead time 
    max_value = sliding_time(data)
    # Set up dummy array for the return period array
    period = np.zeros(np.shape(max_value))
    # # Calculate return period from ERA file
    period = return_period(period, max_value, times)
    x, y = read_coords(file, 'projection_x_coordinate', 'projection_y_coordinate')
    write2nc(period, x[:], y[:], "../py_returnperiod_ens_member%i.nc"%(Scenario))
    
    return

# Slide through the various forecast ensemble members

In [ ]:
# Slider for running through the various ensemble members
@interact(Scenario = (0,11))
calculating_return_periods(Scenario)

## Need to update the plotting